In [1]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

In [2]:
# Imports
import sys
import math
import numpy as np
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
import mpl_toolkits.mplot3d.axes3d as p3

In [3]:
np.random.seed(20)
np.set_printoptions(threshold=sys.maxsize)

In [4]:
%matplotlib inline

# Data

In [5]:
# Read data
path = '../../../dataset_real/gianpaolo/step_0.10/force/forward_accelerations.csv'
motion = np.genfromtxt(path, delimiter=',', dtype=np.float64)
contacts = np.genfromtxt(path, delimiter=',', usecols=(55, 56, 57, 58), dtype=bool)
print("Motion data has a length of ", len(motion))

Motion data has a length of  10257


In [6]:
motion.shape

(10257, 59)

## Height-Force plots

In [7]:
def get_specific_cmd(dataset, fwd, side, rot):
    if abs(fwd): 
        return np.where(dataset[:, 1] > 0)[0] if fwd > 0 else np.where(dataset[:, 1] < 0)[0]
    if abs(side): 
        return np.where(dataset[:, 2] > 0)[0] if side > 0 else np.where(dataset[:, 2] < 0)[0]
    if abs(rot): 
        return np.where(dataset[:, 3] > 0)[0] if rot > 0 else np.where(dataset[:, 3] < 0)[0]

In [8]:
def get_swinging_motions(dataset, height=1):
    # rf min height (i.e swining motion)
    fl_min_height = np.where(dataset[:, 6] < height)[0]
    fr_min_height = np.where(dataset[:, 9] < height)[0]
    rl_min_height = np.where(dataset[:, 12] < height)[0]
    rr_min_height = np.where(dataset[:, 15] < height)[0]
            
    return fl_min_height,fr_min_height,rl_min_height,rr_min_height

# Dataset Preparation

In [9]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [10]:
def yaw_from_quaternion(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    x = Q[0]
    y = Q[1]
    z = Q[2]
    w = Q[3]
     
    return np.arctan2(2 * (w*z + x*y), 1 - 2 * (y*y + z*z))

In [11]:
def quaternion_rotation_matrix(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[3]
    q1 = Q[0]
    q2 = Q[1]
    q3 = Q[2]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

In [12]:
def create_acceleration_com_dataset(dataset, footsteps, motion=None, debug=False):
    inputs = []
    labels = []
    
    for idx, footstep in enumerate(footsteps):
        if idx > 1:
            # Compute time difference between
            # current and next footsteps
            time_difference = abs(dataset[footsteps[idx], 0] - dataset[footsteps[idx-1], 0])
            
            if time_difference < 0.5 and (dataset[footsteps[idx-2], 1] > 0.05 or dataset[footsteps[idx-1], 1] > 0.05):
                lf_rh_moving = contacts[footsteps[idx], 0]
                rf_lh_moving = contacts[footsteps[idx], 1]
                
                if lf_rh_moving == rf_lh_moving:
                    print("Invalid")
                    continue

                # Compute rotation matrices from odometry orientation
                R_prev = quaternion_rotation_matrix(dataset[footsteps[idx-1], 48:52])
                R_curr = quaternion_rotation_matrix(dataset[footsteps[idx], 48:52])

                # Odom base vectors
                prev_base = dataset[footsteps[idx-1], 16:19]
                curr_base = dataset[footsteps[idx], 16:19]

                # Rotate odometry estimation
                rotated_prev_base = prev_base @ R_prev
                rotated_curr_base = curr_base @ R_prev
                base_displacement = rotated_curr_base - rotated_prev_base
                
                prev_yaw = yaw_from_quaternion(dataset[footsteps[idx-1], 48:52])
                curr_yaw = yaw_from_quaternion(dataset[footsteps[idx], 48:52])
                yaw = curr_yaw - prev_yaw
                
                if abs(yaw) > 0.05:
                    continue
                
                # Handle angle wrap
                if np.sign(prev_yaw) != np.sign(curr_yaw):
                    #print("Angle wrap detected.")
                    continue
                    
                inputs.append(dataset[footsteps[idx-2], 1:4].tolist() +
                              dataset[footsteps[idx-1], 1:16].tolist() +
                              dataset[footsteps[idx-1], 19:23].tolist() +
                              [lf_rh_moving, rf_lh_moving])
                
                # Labels
                labels.append([base_displacement[0], base_displacement[1], yaw])
                
    return np.array(inputs, dtype=object), np.array(labels, dtype=object)

# Stack datasets
X_motion_CoM_acceleration, Y_motion_CoM_acceleration = create_acceleration_com_dataset(motion, [x for x in range(len(motion))])
print(X_motion_CoM_acceleration.shape)
print(Y_motion_CoM_acceleration.shape)

(5750, 24)
(5750, 3)


In [13]:
%store X_motion_CoM_acceleration
%store Y_motion_CoM_acceleration

Stored 'X_motion_CoM_acceleration' (ndarray)
Stored 'Y_motion_CoM_acceleration' (ndarray)


In [14]:
min(Y_motion_CoM_acceleration[:, 0])

-0.019032657953404364

In [15]:
for x in range(500):
    print(X_motion_CoM_acceleration[x, :3], X_motion_CoM_acceleration[x, 3:6], Y_motion_CoM_acceleration[x, :])

[0.0 0.0 0.0] [0.1 0.0 0.0] [0.003580319358261974 0.0011485660802389441 0.0013221883273787094]
[0.1 0.0 0.0] [0.1 0.0 0.0] [0.02042476439419083 -0.00037512822232221854 -0.0025591533317495807]
[0.1 0.0 0.0] [0.1 0.0 0.0] [0.03571399754800275 0.0031825013312576367 0.0001157362891392455]
[0.1 0.0 0.0] [0.1 0.0 0.0] [0.03054212941153711 0.0028957677331903353 -0.0013480466721288165]
[0.1 0.0 0.0] [0.1 0.0 0.0] [0.030867042689198154 0.0008899760467470097 0.000516127381446338]
[0.1 0.0 0.0] [0.0 0.0 0.0] [0.020347563166484672 0.00026097598323104876 0.0016082067285241308]
[0.0 0.0 0.0] [0.1 0.0 0.0] [0.019679004560285485 -0.0005084814819635852 -0.00292052885696846]
[0.1 0.0 0.0] [0.1 0.0 0.0] [0.0349443165206762 0.001402128881444531 0.002537788235082522]
[0.1 0.0 0.0] [0.1 0.0 0.0] [0.031574999535664006 0.003602464304292896 -0.0024676976817654267]
[0.1 0.0 0.0] [0.1 0.0 0.0] [0.028837885017254372 -0.0033953377871499546 0.0007960218113074874]
[0.1 0.0 0.0] [0.0 0.0 0.0] [0.02767932737307044 -0.

In [16]:
#for x in range(400):
#    print(f"Prev Velocity: {X_motion_CoM_acceleration[x, -5:-2]}. Curr Velocity: {X_motion_CoM_acceleration[x, :3]}. Displacement: {Y_motion_CoM_acceleration[x, :]}.")